In [ ]:
##in this version i will say screw pipelines and just do it without for now ... 
#maybe I will see what this infinity issue is if I can inspect the resuly of each step manually

##I will now follow Julie Michelman and Zac Stewart's approach to pipelines:
'''Zac does feature union a lot
Julie stays in panda land by creating custom transformers
neither uses column transformers'''

In [ ]:
##Round 1 cleaning complete. v1 decisions: keep all nan as new feature/value; no skew or outlier action has been taken
##Round 1 exploration complete. (a) Distribution of each feature explored (b) feature-target and feature-feature relationships explored 

##Round 1 modelling Plan: make one model for each notebook, try to do systematic tuning ... take best few models (?) or is it too soon?
##then go back and do feature engineering and feature selection based on these base-line models
##Build pipe-line: dummies, scaling, tuning ...

##next steps:
##
##Flask API Deployment
##build small webpage interaction




### Tuning of KNN 

Main parameters: k = num_neighbors, and metric

In [ ]:
## following https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [ ]:
##brief review of KNN vs K-means ... 
##K-means (unspervised) has no labels, just feature space ... you create k-labels (choice of k is tricky and important)
##then run thru k-means clustering algorithm to assign k-labels (clusters) to the data 
##b
##KNN (supervised) is simpler because data already comes with p-labels (the role of k is different here) 
##in the training data on the test data you simply look at the k-nearet neighors (of training set) and take majority vote 
##on what label to assign to new point. The choice of k is important and the metric can change the outcomes.

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score
import seaborn as sns

import pickle

#sns.set()
sns.set(rc={'figure.figsize':(8,5)})

#! very useful 

pd.set_option('display.max_columns', 500)


In [2]:
##baseline models
from sklearn.model_selection import train_test_split # For train/test splits
from sklearn.neighbors import KNeighborsClassifier # The k-nearest neighbor classifier
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV # For optimization


In [3]:

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier

#rfc = RandomForestClassifier(max_depth=4, random_state=0)
#gnb = GaussianNB()
#lgc = LogisticRegression()


In [4]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [5]:
path = "/Users/joejohns/Downloads/data_loans.csv"

data = pd.read_csv(path) 
print(data.shape)
data.head(3)

(614, 13)


Loan_ID Gender Married Dependents Education Self_Employed  \
0  LP001002   Male      No          0  Graduate            No   
1  LP001003   Male     Yes          1  Graduate            No   
2  LP001005   Male     Yes          0  Graduate           Yes   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0         Urban           Y

## data processing from round 1


In [6]:
perc_null(data)

    

Total     Type   Percent
Credit_History        50  float64  0.081433
Self_Employed         32   object  0.052117
LoanAmount            22  float64  0.035831
Dependents            15   object  0.024430
Loan_Amount_Term      14  float64  0.022801
Gender                13   object  0.021173
Married                3   object  0.004886
Loan_ID                0   object  0.000000
Education              0   object  0.000000
ApplicantIncome        0    int64  0.000000
CoapplicantIncome      0  float64  0.000000
Property_Area          0   object  0.000000
Loan_Status            0   object  0.000000

In [7]:
#fix feature names

data.rename(columns = {'ApplicantIncome':'Applicant_Income', 'CoapplicantIncome': 'Coapplicant_Income', 'LoanAmount':'Loan_Amount'}, inplace = True)

In [8]:
#fix target; make it numeric; useful for regression modelling

data.loc[data["Loan_Status"] == 'Y', "Loan_Status"] = 1
data.loc[data["Loan_Status"] == 'N', "Loan_Status"] = 0



In [9]:
#fix this nom cat ... float --> string

data.loc[data['Credit_History'] ==0.0, 'Credit_History'] = 'No'
data.loc[data['Credit_History'] == 1.0, 'Credit_History'] = 'Yes'  #I use capitalized "Yes", "No" to be conistent with their other entries

impute_most_freq =[]

for feat in numer_feats:
    perc_val = data[feat].value_counts()/data.shape[0]
    if perc_val.iloc[0] >=0.7:
        impute_most_freq+=[feat]
    

A = set(ord_cat_feats+numer_feats) 
B = set(nan_feats)
C = set(impute_most_freq)
impute_mean_feats = list(A-C)

In [10]:

nan_feats = perc_null(data)[ perc_null(data)['Total'] >0].index

nan_feats

Index(['Credit_History', 'Self_Employed', 'Loan_Amount', 'Dependents',
       'Loan_Amount_Term', 'Gender', 'Married'],
      dtype='object')

In [11]:

nan_feats = ['Credit_History', 'Self_Employed', 'Gender', 'Married'     # nom_cat ---> Nan = new cat_val (Nans gone check)
          'Loan_Amount',    #record in unk_cat and impute mean
             'Dependents', 'Loan_Amount_Term' ]   #record in unk_cat and impute most_freq 




target = ['Loan_Status']
ID_feats = ['Loan_ID',] 

nan_feats = list(perc_null(data)[ perc_null(data)['Total'] >0].index) 
#nan_feats

#feature types (this must be done manually ... although data types would give a pretty good approximation)

numer_feats = ['Applicant_Income', 'Coapplicant_Income', 'Loan_Amount', 'Loan_Amount_Term',]


ord_cat_feats = [ 'Dependents', 'Education', ]
ord_labels = [ [ '0','1','2','3+' ], ['Not Graduate', 'Graduate']]  #consistent with order of categories in ord_cat_feats (and in df)

nom_cat_feats = ['Gender', 'Married', 'Self_Employed', 'Credit_History', 'Property_Area',]
add_dummy_nan = ['Credit_History','Self_Employed','Gender', 'Married']  #dealing with nom_cat_nan

impute_freq_feats = ['Loan_Amount_Term', 'Dependents' ]  #how can we identify this automatically? It's a cts var where over X% of values = one value 
impute_mean_feats = ['Loan_Amount']     #dealing with nan in numer and ord_cat
 
unk_cat_feats = [ 'Loan_Amount', 'Loan_Amount_Term', 'Dependents' ] #make new category for these 'unk' = yes/no; ##impute by mean
#tracking nans which were lost by imputation 


scale_feats = ord_cat_feats+numer_feats
dummies_feats = ['Property_Area','missing_data']+['Credit_History', 'Self_Employed', 'Gender', 'Married',]  #regular dummies



In [ ]:
#change all nan to 'unk' in the categoricals ...
#for feat in set(nan_feats)&set(dummies_feats):
#    data.loc[data[feat].isnull(),  feat ] = 'unk'

##eventually perhaps we can make this into a class that can be used in a pipe line ... 

#step 1. create new categorical (records if any null values in any of specified cats)

def update_missing(x, feature):
    if x == 'No':
        return feature
    else: 
        return x+'&'+feature

data['missing_data'] = 'No'  
for feature in unk_cat_feats:
    data.loc[(data[feature].isnull()) , 'missing_data'] = data.loc[(data[feature].isnull()) , 'missing_data'].map(lambda x: update_missing(x, feature))

    
nom_cat_feats+=['missing_data']

In [ ]:
#testing len(data.loc[(data['Education'].isnull()), 'Education'])

In [16]:
unk_cat_dummies = ['missing_'+x for x in unk_cat_feats]

In [17]:
unk_cat_dummies

['missing_Loan_Amount', 'missing_Loan_Amount_Term', 'missing_Dependents']

In [12]:
for feature in unk_cat_feats:
    if len(data.loc[(data[feature].isnull()), :].count()) > 0:
        data['missing_'+feature] = 0  #dummy var
        data.loc[(data[feature].isnull()) , 'missing_'+feature] = 1

In [13]:
for feature in unk_cat_feats:
    print(data['missing_'+feature].value_counts())

0    592
1     22
Name: missing_Loan_Amount, dtype: int64
0    600
1     14
Name: missing_Loan_Amount_Term, dtype: int64
0    599
1     15
Name: missing_Dependents, dtype: int64


##now we execute all imputing and scaling ... bruh


#steps:

#1. DONE create new cat recording if NaN in specified numeric or nominal features 
#(do this before imputing of course)

#numeric and nominal: 
#2. impute most_freq (Loan_Term)
#3. impute mean (everyone else)
#4. scale by max, min (or by mean; try both)

#nominal cats:
#5. get_dummies, nan yes
#6. get_dummies, nan no

#perc_null(data)

In [15]:
set(data.columns)

{'Applicant_Income',
 'Coapplicant_Income',
 'Credit_History',
 'Dependents',
 'Education',
 'Gender',
 'Loan_Amount',
 'Loan_Amount_Term',
 'Loan_ID',
 'Loan_Status',
 'Married',
 'Property_Area',
 'Self_Employed',
 'missing_Dependents',
 'missing_Loan_Amount',
 'missing_Loan_Amount_Term'}

In [ ]:
model_feats = list(set(data.columns+unk_cat_dummies) - set(target+ID_feats))


In [ ]:
model_feats

In [ ]:
##now let's make a pipe line ... first with the normal skl functions ... then with my functions 

In [ ]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler



preprocessor = ColumnTransformer([
('impute_most_freq', impute_most_freq, impute_freq_feats), 
('impute_mean', impute_mean, impute_mean_feats),     
('ordinals', ordinals, ord_cat_feats),
])

Steps I will follow

- make new dummy_vars for the non nom_cat missing NaN  #this tracks all missing NaN for us ... is it worth all the dummies? Make numeric/ordinal?
- ordinal encode (NaN just leave for now; impute plus recorded in new_cat ... does not make sense to make ordinal Nan ... -1? no)
- dummies (Nan ---> new cat val) # must come after make new cat ... 
- impute (freq = ord plus numer1; mean numer2)  # must come after ordinal encode 
- scale (numer and ord ... X ---> X/|max X|).   # I think X --> X - X_min is an unneccsary distortion



In [ ]:

class DFOrdinalEncoder(TransformerMixin):  #transformerMixin just gives us fit_transform automatically
    # FunctionTransformer but for pandas DataFrames

    def __init__(self, *args, **kwargs):
        self.ft = FunctionTransformer(*args, **kwargs)

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        Xt = self.ft.transform(X)
        Xt = pd.DataFrame(Xt, index=X.index, columns=X.columns)
        return Xt

In [ ]:
#let me now follow https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/
#which does cv = 3 instead of cv = stratififedkfold( ... )


y = data_mean.loc[:, target]
X= data_mean.loc[:, model_feats]
y = y.astype(int)
# Split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(
    X,  
    y, 
    test_size=1/3,
    random_state=0)
 
print(X_train.shape)
print(X_test.shape)

#np.array(X)

In [ ]:
X_train

In [ ]:
knn = KNeighborsClassifier()

knn.fit(X_train, np.ravel(y_train))

In [ ]:
knn.score(X_test, np.ravel(y_test))  #77% ? shitty! ha ha was 80% in my original version

In [ ]:
##Sat Sept 18, 1:30 pm ... make pipe line? or do tuning and feature selection? 

In [ ]:

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=4, random_state=0)
gnb = GaussianNB()
lgc = LogisticRegression()


In [ ]:

rfc.fit(X_train, np.ravel(y_train))

In [ ]:
rfc.score(X_test, np.ravel(y_test))  ##aha! I used rfc before and it IS improved to 82% ... cool. That makes me feel better ... (all that effort should not make it worse)

In [ ]:

lgc.fit(X_train, np.ravel(y_train))

In [ ]:
lgc.score(X_test, np.ravel(y_test))  ##aha! I used rfc before and it IS improved to 82% ... cool. That makes me feel better ... (all that effort should not make it worse)